In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import sys
import datetime
import math

ROOT_DIR = '../'
sys.path.insert(1, '../production_code/')
from constants import *

In [ ]:
area_type = 'region'

df_train = pd.read_csv(ROOT_DIR + ('_' + area_type +'.').join(TESTING_DATA_DIR.split('.')))
df_test = pd.read_csv(ROOT_DIR + ('_' + area_type +'.').join(TESTING_DATA_DIR.split('.')))

In [ ]:
df_train

# basic machine learning model

In [ ]:


# # Encode categorical features
# le = LabelEncoder()
# df_train["day"] = le.fit_transform(df_train["day"])
# df_test["day"] = le.transform(df_test["day"])
# le = LabelEncoder()
# df_train["hour_bin"] = le.fit_transform(df_train["hour_bin"])
# df_test["hour_bin"] = le.transform(df_test["hour_bin"])
# le = LabelEncoder()
# df_train["region"] = le.fit_transform(df_train["region"])
# df_test["region"] = le.transform(df_test["region"])
# le = LabelEncoder()
# df_train["sky"] = le.fit_transform(df_train["sky"])
# df_test["sky"] = le.transform(df_test["sky"])


# One hot encoding
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)


# df_train[["Ambulance", "Police"]] = df_train[["Ambulance", "Police"]].applymap(lambda x: math.log(x) if x > 0 else pd.NA)
# df_test[["Ambulance", "Police"]] = df_test[["Ambulance", "Police"]].applymap(lambda x: math.log(x) if x > 0 else pd.NA)

df_train = df_train.dropna()
df_test = df_test.dropna()


scale_y = max(df_train[["Ambulance", "Police"]].max().max(), 10)
print('scaling y by: ' + str(scale_y))

# Split data into features and labels
x_train = df_train.drop(["Ambulance", "Police"], axis=1)
y_train = df_train[["Ambulance", "Police"]] / scale_y
x_test = df_test.drop(["Ambulance", "Police"], axis=1)
y_test = df_test[["Ambulance", "Police"]] / scale_y


In [ ]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
import numpy as np


# Print the shape and type of the data
print(x_train.shape) # (60000, 28, 28)
print(y_train.shape) # (60000,)
print(x_test.shape) # (10000, 28, 28)
print(y_test.shape) # (10000,)


In [ ]:
x_train = x_train.applymap(pd.to_numeric)
y_train = y_train.applymap(pd.to_numeric)
x_test = x_test.applymap(pd.to_numeric)
y_test = y_test.applymap(pd.to_numeric)

In [ ]:
# y_train = y_train.applymap(lambda x: math.log(x*100))
# y_test = y_test.applymap(lambda x: math.log(x*100))
y_train

# Very crude experimentation in deep learning

In [ ]:

# Create a model
model = keras.Sequential([
  keras.layers.Dense(20, activation='relu'),
  keras.layers.Dropout(0.05),
  keras.layers.Dense(30, activation='relu'),
  keras.layers.Dropout(0.1),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(30, activation='tanh'),
  keras.layers.Dense(10, activation='tanh'),
  keras.layers.Dense(2, activation='linear')
])


model.compile(optimizer='Adam', loss='mean_squared_error', metrics=['mean_squared_error', 'accuracy'])

model.fit(x_train, y_train, batch_size = 2, epochs=10)



# Plotting results on testing data

In [ ]:
import math 
from sklearn.metrics import mean_squared_error

# Evaluate the model
model.evaluate(x_test, y_test)

y_pred = pd.DataFrame(model.predict(x_test), columns = ['Predicted Police','Predicted Ambulance'])
y_comb = pd.concat(
            [y_pred, y_test], 
            axis = 1
        ) * scale_y     # unscaling y

# renaming columns
y_comb = y_comb.rename(
    columns = {
        'Ambulance': 'True Ambulance', 
        'Police': 'True Police'
    })

rmse = mean_squared_error(y_comb['True Ambulance'], y_comb['Predicted Ambulance'], squared=False) 
print('Ambulance RMSE: ' + str(rmse))
rmse = mean_squared_error(y_comb['True Police'], y_comb['Predicted Police'], squared=False) 
print('Police RMSE: ' + str(rmse))

# widening data for plotting
y_comb_wide = pd\
    .wide_to_long(
        y_comb.reset_index(), 
        stubnames=['Predicted', 'True'], 
        i='index', 
        j='Emergency Type', 
        sep=' ', 
        suffix='\w+')\
    .reset_index()\
    .drop('index', axis = 1)\
    .dropna()

# plotting data
fig = px.scatter(
    y_comb_wide,
    x = 'True',
    y = 'Predicted',
    color = 'Emergency Type', symbol = 'Emergency Type'
    )

xmax = max(y_comb_wide['True'])
line = { 'type': 'line', 'x0': 0, 'y0': 0, 'x1': xmax, 'y1': xmax, 'line': { 'color': 'black', 'width': 1 } }
fig.update_layout(shapes=[line])
fig.show()